In [1]:
import datetime
import pandas as pd
import plotly.express as px
from greykite.framework.templates.autogen.forecast_config import ForecastConfig, MetadataParam, ModelComponentsParam
from greykite.framework.benchmark.data_loader_ts import DataLoaderTS
from greykite.framework.templates.forecaster import Forecaster


/Users/Edouard_1/.pyenv/versions/3.10.6/envs/PDPA-virt_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/Users/Edouard_1/Downloads/cluster1_Ouest_new_atmo.csv')
df

,Date_time,PM25,PM10,NO2,O3,Pollution_peak,ATMO,sin_Month,cos_Month,sin_day,cos_day,confinement
0,2018-01-01,12.90,8.74,14.33,78.50,0,1,0.000000,1.000000,-0.781831,0.623490,0
1,2018-01-02,12.90,13.56,34.67,71.00,0,1,0.000000,1.000000,0.000000,1.000000,0
2,2018-01-03,12.90,12.80,24.25,80.50,0,1,0.000000,1.000000,0.781831,0.623490,0
3,2018-01-04,12.90,9.09,25.25,69.00,0,1,0.000000,1.000000,0.974928,-0.222521,0
4,2018-01-05,12.79,15.81,46.75,79.50,0,1,0.000000,1.000000,0.433884,-0.900969,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1794,2022-11-30,17.67,31.65,46.58,42.15,0,1,-0.866025,0.500000,0.781831,0.623490,0
1795,2022-12-01,22.85,32.04,38.62,25.10,0,2,-0.500000,0.866025,0.974928,-0.222521,0
1796,2022-12-02,28.70,35.24,39.72,24.95,0,3,-0.500000,0.866025,0.433884,-0.900969,0
1797,2022-12-03,19.83,23.61,32.00,13.55,0,1,-0.500000,0.866025,-0.433884,-0.900969,0


In [4]:
source_dir= "/Users/Edouard_1/code/TheLab75/ParisDeepAirProject/Model_greykite"

forecaster = Forecaster()

forecaster.load_forecast_result(
    source_dir,
    load_design_info=True)

forecaster

In [8]:
result = forecaster.forecast_result
result




ForecastResult(timeseries=<greykite.framework.input.univariate_time_series.UnivariateTimeSeries object at 0x13373b2e0>, grid_search=RandomizedSearchCV(cv=RollingTimeSeriesSplit(expanding_window=True, forecast_horizon=7,
            max_splits=3, min_train_periods=14, periods_between_splits=7,
            periods_between_train_test=0, use_most_recent_splits=None),
                   estimator=Pipeline(steps=[('input',
                                              PandasFeatureUnion(transformer_list=[('date',
                                                                                    Pipeline(steps=[('select_date',
                                                                                                     ColumnSelector(column_names=['ts']))...
                            'OutsideTolerance3p': make_scorer(score_func_finite),
                            'OutsideTolerance4p': make_scorer(score_func_finite),
                            'OutsideTolerance5p': make_scorer(scor

In [9]:
df_predict_and_actual= result.forecast.df
df_predict_and_actual



,Date_time,actual,forecast,forecast_lower,forecast_upper
0,2018-01-01,0,NaN,NaN,NaN
1,2018-01-02,0,-0.000273,-1.237044,1.236497
2,2018-01-03,0,-0.000318,-1.220263,1.219627
3,2018-01-04,0,-0.000374,-1.201665,1.200917
4,2018-01-05,0,-0.000425,-1.190566,1.189717
...,...,...,...,...,...
1794,2022-11-30,0,0.159300,-1.229929,1.548528
1795,2022-12-01,1,0.175868,-1.237421,1.589156
1796,2022-12-02,2,0.184012,-1.236787,1.604811
1797,2022-12-03,0,0.187947,-1.235639,1.611532


In [10]:
df_predict_and_actual = df_predict_and_actual[1:] #remove first row with NA
df_predict_and_actual


,Date_time,actual,forecast,forecast_lower,forecast_upper
1,2018-01-02,0,-0.000273,-1.237044,1.236497
2,2018-01-03,0,-0.000318,-1.220263,1.219627
3,2018-01-04,0,-0.000374,-1.201665,1.200917
4,2018-01-05,0,-0.000425,-1.190566,1.189717
5,2018-01-06,0,-0.000483,-1.179397,1.178431
...,...,...,...,...,...
1794,2022-11-30,0,0.159300,-1.229929,1.548528
1795,2022-12-01,1,0.175868,-1.237421,1.589156
1796,2022-12-02,2,0.184012,-1.236787,1.604811
1797,2022-12-03,0,0.187947,-1.235639,1.611532


In [12]:
import numpy as np


In [13]:
#get a sf with class as integer
seven_days_predicted = pd.DataFrame(np.round(df_predict_and_actual['forecast']).astype(int))[-7:] #remove 7 rows forecasts

#reset indef to get day as column
seven_days_predicted.reset_index(inplace=True, drop=True) # get the good number for days
seven_days_predicted.reset_index(inplace=True, drop=None) # put is as columns 
seven_days_predicted.columns = ['days', 'forecast'] # rename columns
seven_days_predicted



,days,forecast
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0


In [17]:
y_true = df_predict_and_actual['actual'][:-7].astype(int)
y_true





1       0
2       0
3       0
4       0
5       0
       ..
1787    0
1788    0
1789    0
1790    0
1791    0
Name: actual, Length: 1791, dtype: int64

In [18]:
y_pred = np.round(df_predict_and_actual['forecast'][:-7]).astype(int)
y_pred


1       0
2       0
3       0
4       0
5       0
       ..
1787    0
1788    0
1789    0
1790    0
1791    0
Name: forecast, Length: 1791, dtype: int64

In [19]:
results_df = pd.DataFrame({"actual": y_true,
                           "predicted": y_pred})

confusion_matrix = pd.crosstab(index= results_df['actual'],
                               columns = results_df['predicted'], normalize='index')


In [20]:
import seaborn as sns
sns.heatmap(confusion_matrix, annot=True, fmt=".0", linewidths = 0.01,);
